In [1]:
# Import necessary libraries
from datasets import load_dataset
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


print("GPU Available:", tf.config.list_physical_devices('GPU'))




GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Load the Tiny ImageNet dataset
cache_dir = "/kaggle/working/tiny-imagenet"  # Kaggle's working directory

# Load the dataset (it will download if not already present)
ds = load_dataset("zh-plus/tiny-imagenet", cache_dir=cache_dir)

# Filter the dataset to include only the first 20 classes for the training set
filtered_ds_train = ds['train'].filter(lambda example: example['label'] < 20)
filtered_ds_valid = ds['valid'].filter(lambda example: example['label'] < 20)  # Use valid as validation set



README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

(…)-00000-of-00001-1359597a978bc4fa.parquet:   0%|          | 0.00/146M [00:00<?, ?B/s]

(…)-00000-of-00001-70d52db3c749a935.parquet:   0%|          | 0.00/14.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Load the MobileNetV2 model pre-trained on ImageNet, without the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a custom dense layer to create embeddings of a specific size
embedding_size = 64
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(embedding_size, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)  # Only 20 classes
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model for training
model.compile(optimizer=Adam(),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

# Function to preprocess images
def preprocess_image(image):
    image = Image.fromarray(image)
    image = image.resize((224, 224))
    image = np.array(image)
    if image.shape == (224, 224, 3):  # Ensure the image has 3 channels
        return preprocess_input(image)
    else:
        print(f"Skipping image with shape {image.shape}")
        return None

# Function to prepare data
def prepare_data(dataset):
    images, labels = [], []
    for example in dataset:
        processed_image = preprocess_image(np.array(example['image']))
        if processed_image is not None:
            images.append(processed_image)
            labels.append(example['label'])
    return np.array(images), np.array(labels)

# Prepare the training data
train_images, train_labels = prepare_data(filtered_ds_train)

# Prepare the validation data (using the valid dataset)
valid_images, valid_labels = prepare_data(filtered_ds_valid)

# Define a callback to save the model with the best validation accuracy
checkpoint = ModelCheckpoint(
    '/kaggle/working/mobilenetv2_best_model.keras',  # Path where the best model will be saved
    monitor='val_sparse_categorical_accuracy',  # Monitor validation accuracy
    save_best_only=True,  # Save only when the validation accuracy improves
    mode='max',  # 'max' means we want to maximize the validation accuracy
    verbose=1  # To print when a new best model is saved
)

# Train the model on the Kaggle GPU with validation data, using the callback
history = model.fit(
    train_images, train_labels, 
    epochs=300, batch_size=64, 
    validation_data=(valid_images, valid_labels),
    callbacks=[checkpoint]  # Include the checkpoint callback in the training process
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape (224, 224)
Skipping image with shape 

I0000 00:00:1731347051.463288      84 service.cc:145] XLA service 0x7be6c4004b90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731347051.463348      84 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  1/156 ━━━━━━━━━━━━━━━━━━━━ 1:57:38 46s/step - loss: 3.1092 - sparse_categorical_accuracy: 0.0938

I0000 00:00:1731347073.766939      84 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - loss: 1.7368 - sparse_categorical_accuracy: 0.5047
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.12738, saving model to /kaggle/working/mobilenetv2_best_model.keras
156/156 ━━━━━━━━━━━━━━━━━━━━ 95s 319ms/step - loss: 1.7346 - sparse_categorical_accuracy: 0.5053 - val_loss: 13.1564 - val_sparse_categorical_accuracy: 0.1274
Epoch 2/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.8111 - sparse_categorical_accuracy: 0.7671
Epoch 2: val_sparse_categorical_accuracy improved from 0.12738 to 0.15948, saving model to /kaggle/working/mobilenetv2_best_model.keras
156/156 ━━━━━━━━━━━━━━━━━━━━ 24s 152ms/step - loss: 0.8112 - sparse_categorical_accuracy: 0.7670 - val_loss: 14.2047 - val_sparse_categorical_accuracy: 0.1595
Epoch 3/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.5887 - sparse_categorical_accuracy: 0.8229
Epoch 3: val_sparse_categorical_accuracy did not improve from 0.15948
156/156 ━━━━━━━━━━━━━━━━━━━━ 23s 

In [ ]:
# Save the model locally on Kaggle's output directory
model_without_last_layer = Model(inputs=model.input, outputs=model.layers[-2].output)
model_without_last_layer.save('/kaggle/working/mobilenetv2_embeddings_model.h5')

# Optionally, plot the training and validation accuracy and loss
plt.plot(history.history['sparse_categorical_accuracy'], label='Train Accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Save the model locally on Kaggle's output directory
model_without_last_layer = Model(inputs=model.input, outputs=model.layers[-2].output)
model_without_last_layer.save('/kaggle/working/mobilenetv2_embeddings_model.h5')